# There is a problem 😧😧😧😧😧😧
# What is it ?????? & What we should dooo???


**The input:**

it representing Reddit Fake Post Detection ( by looking at its title)

conisting of only one feature that represent the title of the posts



**The output:**

it is the label which specify if the post in Reddit is fack or not( prediction of the probability (0-1, float))


**Data mining function:**

1.Generalization and Summarization

2.Association and Correlation

3.Clustering

4.Outlier/Anomaly Analysis

5.Classification & Prediction

5.Time and Ordering 

6.Structure and Network Analysis

**The data mining in this problem requires Classification & Prediction After cleaning the data by:**

*   handeling any missing data
*   removing useless data


**The impact**
if we use the data without cleaning,it will give us a model with low accuracy which will not learn well.

The real-life impact of this model is represent in limitting the number of fack posts that could be created on Reddit site by detecting it and distinguish it from the real ones just by looking at the title.


if we solve it, we will destroy the fack messages

**The challenges:**

* Noise data (useless data)
* Missing data
* label that contain niether 1 nor 0
* the preprocessing of the data


**The ideal solution:**

1. clean and preprocess the data 
2. we will try word and character vectorizer and see which one will give high performance 
3. we found that word vectorizer will give best performance
4. trying different models with different hyperparaameters to get the ideal solution
5. the best model that gives higher performance score was LogisticRegression with validation set followed by LogisticRegression with word vectorizer then XGBClassifier on word with_validation_set

**The experimental protocol used and how was it carried out:**
After loading the data and cleaning and preprocessing it, different experimental protocol are used through different trials:

using validation set to measure the performance while using any classifier with word-level vectorizer
using hyperparamter search method RandomizedSearchCV / or BayesSearchCV

we measure the perormance using (roc_auc)

**The preprocessing steps are used:**
* we remove the labels with values of not 0 or 1
* we apply the character and the word using stemmer(we remove the lower case,remove the stopping word,remove single letter chars,
convert all whitespaces to single wspace, punctuation and stemm, remove html tags, remove text that contain very low number of characters (consider it useless data), Keep only ASCII + European Chars and whitespace, no digits
* apply feature creation with TF-IDF (word vectorizer & character vectorizer)
* we try the the apply again on the character and word but this time we use the lemmatizer (with the same steps)

# Import the libraries and packages

In [ ]:
!pip install scikit-optimize

In [ ]:
import re
import pickle
import sklearn
import pandas as pd
import numpy as np
import holoviews as hv
import nltk 
from bokeh.io import output_notebook
output_notebook()

from pathlib import Path

# some seeting for pandas and hvplot

pd.options.display.max_columns = 100
pd.options.display.max_rows = 300
pd.options.display.max_colwidth = 100
np.set_printoptions(threshold=2000)

Loading BokehJS ...

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
%matplotlib inline
import pandas as pd
# import matplotlib.pyplot as plt
import numpy as np
# import seaborn as sns

# **Data Cleaning and Preprocessing**

# Load the data(train & test) and read it

In [ ]:
#Reading our dataset
df_train = pd.read_csv("xy_train.csv")  #Reading our train set 
df_test = pd.read_csv("x_test.csv")   #Reading our test set
df_train #printing our train set

,id,text,label
0,265723,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0
1,284269,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0
2,207715,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0
...,...,...,...
59995,70046,"Finish Sniper Simo H盲yh盲 during the invasion of Finland by the USSR (1939, colorized)",0
59996,189377,"Nigerian Prince Scam took $110K from Kansas man; 10 years later, he's getting it back",1
59997,93486,Is It Safe To Smoke Marijuana During Pregnancy? You鈥檇 Be Surprised Of The Answer | no,0
59998,140950,Julius Caesar upon realizing that everyone in the room has a knife except him (44 bc),0


# Remove the id as it is not a feature



In [ ]:
df_test = pd.read_csv("x_test.csv")   #Reading our test set

df_test_id=df_test['id']
df_test_id
df_test.drop(columns=['id'],axis='columns', inplace=True)

In [ ]:
df_train.drop(columns=['id'],axis='columns', inplace=True)

In [ ]:
df_train.info() #to get the information of the data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    60000 non-null  object
 1   label   60000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 937.6+ KB


Calculate the nulls to know if there is any missing data

In [ ]:
df_train.isna().sum()

text     0
label    0
dtype: int64

Here ,we want only the 0s & 1s

In [ ]:
# Drop rows that contain label = 2 
df_train = df_train[(df_train["label"] != 2)]
df_train

,text,label
0,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0
1,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0
2,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0
3,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0
4,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0
...,...,...
59995,"Finish Sniper Simo H盲yh盲 during the invasion of Finland by the USSR (1939, colorized)",0
59996,"Nigerian Prince Scam took $110K from Kansas man; 10 years later, he's getting it back",1
59997,Is It Safe To Smoke Marijuana During Pregnancy? You鈥檇 Be Surprised Of The Answer | no,0
59998,Julius Caesar upon realizing that everyone in the room has a knife except him (44 bc),0


# Apply stemmer

There is a rule-based process of word form conversion where word-suffixes are truncated irrespective of whether the root word is an actual word in the language dictionary so we applied stemming to convert tokens to their root form
        

steps:

---




        * remove any html tags (< /br> often found)

        * Keep only ASCII + European Chars and whitespace, no digits

        * remove single letter chars

        * convert all whitespaces (tabs etc.) to single wspace
           if not for embedding (but e.g. tdf-idf):

        * all lowercase

        * remove stopwords, punctuation and stemm

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words("english"))


def clean_text(text, for_embedding=False):
    """ steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [
            stemmer.stem(word) for word in words_tokens_lower if word not in stop_words
        ]

    text_clean = " ".join(words_filtered)
    return text_clean

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LAB\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LAB\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


we want to clean the comments from the train data 

In [ ]:
# Clean Comments
df_train["text_clean"] = df_train.loc[df_train["text"].str.len() > 20, "text"]
df_train["text_clean"] = df_train["text_clean"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x
)

C:\Users\LAB\AppData\Local\Temp\ipykernel_15128\3569877927.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["text_clean"] = df_train.loc[df_train["text"].str.len() > 20, "text"]
C:\Users\LAB\AppData\Local\Temp\ipykernel_15128\3569877927.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["text_clean"] = df_train["text_clean"].map(


Here, we drop any nulls or spaces

In [ ]:
# Drop when any of x missing
df_train = df_train[(df_train["text_clean"] != "") & (df_train["text_clean"] != "null")]

df_train = df_train.dropna(axis="index", ).reset_index(drop=True)

Show the data after removing allllllll of the previous noise 😠😠(after preprocessing)

In [ ]:
df_train

,text,label,text_clean
0,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0,group friend began volunt homeless shelter neighbor protest see anoth person also need natur lik...
1,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0,british prime minist theresa may nerv attack former russian spi govern conclud high like russia ...
2,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0,goodyear releas kit allow ps brought heel https youtub com watch alxulk cg zwillc fish midatlant...
3,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0,happi birthday bob barker price right host like rememb man said ave pet spay neuter fuckincorpor...
4,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0,obama nation innoc cop unarm young black men die magic johnson jimbobshawobodob olymp athlet sho...
...,...,...,...
59753,"Finish Sniper Simo H盲yh盲 during the invasion of Finland by the USSR (1939, colorized)",0,finish sniper simo yh invas finland ussr color
59754,"Nigerian Prince Scam took $110K from Kansas man; 10 years later, he's getting it back",1,nigerian princ scam took kansa man year later get back
59755,Is It Safe To Smoke Marijuana During Pregnancy? You鈥檇 Be Surprised Of The Answer | no,0,safe smoke marijuana pregnanc surpris answer
59756,Julius Caesar upon realizing that everyone in the room has a knife except him (44 bc),0,julius caesar upon realiz everyon room knife except bc


We will apply the same thing on the test data

In [ ]:
# Clean texts for test data
df_test["text_clean"] = df_test.loc[df_test["text"].str.len() > 0, "text"]
df_test["text_clean"] = df_test["text_clean"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x
)

In [ ]:
df_test

,text,text_clean
0,stargazer,stargaz
1,yeah,yeah
2,PD: Phoenix car thief gets instructions from YouTube video,pd phoenix car thief get instruct youtub video
3,"As Trump Accuses Iran, He Has One Problem: His Own Credibility",trump accus iran one problem credibl
4,"""Believers"" - Hezbollah 2011",believ hezbollah
...,...,...
59146,Bicycle taxi drivers of New Delhi,bicycl taxi driver new delhi
59147,Trump blows up GOP's formula for winning House races,trump blow gop formula win hous race
59148,"Napoleon returns from his exile on the island of Elba. (March 1815), Colourised",napoleon return exil island elba march colouris
59149,Deep down he always wanted to be a ballet dancer,deep alway want ballet dancer


In [ ]:
df_train_copy=df_train.copy()

We count the words (the frequency of it)

In [ ]:
from bokeh.models import NumeralTickFormatter
# Word Frequency of most common words
word_freq = pd.Series(" ".join(df_train_copy["text_clean"]).split()).value_counts()
word_freq[1:40]

one         3285
like        3128
new         2998
look        2847
color       2737
man         2729
get         2602
trump       2578
say         2347
peopl       2316
use         2307
first       2248
make        2227
old         2226
time        2027
poster      2000
found       1999
day         1935
war         1858
post        1648
world       1570
work        1531
show        1513
us          1506
american    1504
take        1491
life        1482
psbattl     1470
help        1442
go          1420
state       1409
back        1369
two         1364
school      1345
see         1329
photo       1324
made        1314
right       1311
save        1308
dtype: int64

list most uncommon words

In [ ]:
word_freq[-20:].reset_index(name="freq")

,index,freq
0,robertshaw,1
1,sawzal,1
2,vle,1
3,battersea,1
4,flung,1
5,sekou,1
6,hiroshig,1
7,dirtier,1
8,noseslid,1
9,danta,1


we just check the missing after preprocessing(on train & test)

In [ ]:
#Checking missing values
df_train_copy.isnull().sum()

text          0
label         0
text_clean    0
dtype: int64

In [ ]:
#Checking missing values
df_test.isnull().sum()

text          0
text_clean    0
dtype: int64

# Spliting data to X & Y then print the shape

In [ ]:
# Sample data - 25% of data to test set
x_train = df_train_copy["text_clean"]
y_train = df_train_copy["label"]
x_test=df_test["text_clean"]

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(59758,)
(59758,)
(59151,)


# Split data to train & validation

In [ ]:
x_train1, x_val, y_train1, y_val = train_test_split(x_train, y_train, train_size = 0.8, stratify = y_train, shuffle=True, random_state = 1234)

Create a list where train data indices are -1 and validation data indices are 0

use the list to create PredefinedSplit(for validation set as we will use it sooner ⁉)

In [ ]:
%%time
from sklearn.model_selection import PredefinedSplit

# X_tr (new training set), X_train
split_index = [-1 if x in x_train1.index else 0 for x in x_train.index]
pd = PredefinedSplit(test_fold = split_index)

CPU times: total: 46.9 ms
Wall time: 44.6 ms


# **Apply fleature creation with TF-IDF** 

as classification models cannot deal with text data directly, we need to convert our it to a numeric representation. so, we will see it using tfidf


# XGB Classifier with character vectorization using BayesSearchCV


In [ ]:
from skopt import BayesSearchCV

In [ ]:
%%time
pipe = Pipeline([
                 ("tfidf", TfidfVectorizer(analyzer="char")),
                 ('my_classifier', XGBClassifier())
                 ])

params = {
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
    'my_classifier__n_estimators': [100, 150, 200],  
    'my_classifier__max_depth':[300, 500, 1000]
}


pipe_clf = BayesSearchCV(
    pipe, params, cv=5, verbose=1, n_jobs=2,
    n_iter=50,
    scoring='roc_auc')

pipe_clf.fit(x_train, y_train)

print('best score {}'.format(pipe_clf.best_params_))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
best score OrderedDict([('my_classifier__max_depth', 1000), ('my_classifier__n_estimators', 200), ('tfidf__max_df', 0.3), ('tfidf__min_df', 64)])
CPU times: total: 4min 11s
Wall time: 7min 40s


* we make a pipline contains {our vectorizer:tfidf (applid on the character),and the model(classifier):XGBClassifier}
* we played wuth the hyperparameters :
1. max_df->tfidf
2. min_df->tfidf
3. no. of estimator->classifier
4. max_depth ->classifier
* in the BayesSearchCV -> {crossvalidation=5, no. of itreration =50, and our metric ->roc_auc}



the best score 
1. 'my_classifier__max_depth'-> 1000
2. 'my_classifier__n_estimators'-> 200
3. 'tfidf__max_df'-> 0.3
4. 'tfidf__min_df'-> 64

we print the best hyperparameters

In [ ]:
print('best score {}'.format(pipe_clf.best_score_))

best score 0.5310297304897816


# Conclision:
 XGBClassifier on character vectorization is toooooo baddddd 😞😞😞😞😞😞

In [ ]:
import pandas as pand
submission = pand.DataFrame()
submission['id'] = df_test_id

submission['label'] = pipe_clf.predict_proba(df_test['text_clean'])[:,1]

submission.to_csv('baysien_search_xg_ch.csv', index=False)

# XGB Classifier with word vectorization using BayesSearchCV


In [ ]:
%%time
pipe1 = Pipeline([
                 ("tfidf", TfidfVectorizer(analyzer="word",ngram_range=(1, 3), norm="l2")),
                 ('my_classifier', XGBClassifier())
                 ])

params1 = {
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
    'my_classifier__n_estimators': [100, 200],  
    'my_classifier__max_depth':[200, 300]
}


pipe_clf1 = BayesSearchCV(
    pipe1, params1, cv=3, verbose=1, n_jobs=1,
 
    n_iter=10,
    scoring='roc_auc')

pipe_clf1.fit(x_train, y_train)


print('best score {}'.format(pipe_clf1.best_params_))

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
best score OrderedDict([('my_classifier__max_depth', 248), ('my_classifier__n_estimators', 184), ('tfidf__max_df', 0.3), ('tfidf__min_df', 21)])
CPU times: total: 5h 7min 40s
Wall time: 56min


* we make a pipline contains {our vectorizer:tfidf (applid on the word),and the model(classifier):XGBClassifier}
* we played wuth the hyperparameters :
1. max_df->tfidf
2. min_df->tfidf
3. no. of estimator->classifier
4. max_depth ->classifier
* in the BayesSearchCV -> {crossvalidation=3, no. of itreration =10, and our metric ->roc_auc}

best score OrderedDict
1. 'my_classifier__max_depth'-> 248
2. 'my_classifier__n_estimators'-> 184
3. 'tfidf__max_df'-> 0.3
4. 'tfidf__min_df'-> 21

we print the best hyperparameters

In [ ]:
print('best score {}'.format(pipe_clf1.best_score_))

best score 0.8289744592700323


# Conclusion:
XGBClassifier on word vectorization is much moreeeeeeee better than on character 🤭🤭🤭🤭🤭🤭

In [ ]:
import pandas as pand
submission = pand.DataFrame()
submission['id'] = df_test_id

submission['label'] = pipe_clf1.predict_proba(df_test['text_clean'])[:,1]

submission.to_csv('baysien_search_xg_wd.csv', index=False)

# XGB Classifier with validation set on word using RandomSearchCV


we will use our predefined split internally to determine which sample belongs to the validation set

In [ ]:
pipe_val = Pipeline([
                 ("tfidf", TfidfVectorizer(analyzer="word")),
                 ('my_classifier', XGBClassifier())
                 ])

params_val = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
    'my_classifier__n_estimators': [100, 150, 200],  
    'my_classifier__max_depth':[200, 300, 500]
}
pipe_clf_val = RandomizedSearchCV(
    pipe_val, params_val, cv=pd, verbose=1, n_jobs=2,
    n_iter=5,
    scoring='roc_auc')

pipe_clf_val.fit(x_train, y_train)

print('best score {}'.format(pipe_clf_val.best_params_))

Fitting 1 folds for each of 5 candidates, totalling 5 fits
best score {'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 15, 'tfidf__max_df': 0.3, 'my_classifier__n_estimators': 150, 'my_classifier__max_depth': 200}


* we make a pipline contains {our vectorizer:tfidf (applid on the validation set on word),and the model(classifier):XGBClassifier}
* we played wuth the hyperparameters :
1. max_df->tfidf
2. min_df->tfidf
3. no. of estimator->classifier
4. max_depth ->classifier
5. ngram_range_df->tfidf
* in the RandomizedSearchCV -> {crossvalidation=pd(which the predifined function that we did before for
 validation set), no. of itreration =5, and our metric ->roc_auc}

best score 
1. 'tfidf__ngram_range'-> (1, 2)
2. 'tfidf__min_df'-> 15
3. 'tfidf__max_df'-> 0.3
4. 'my_classifier__n_estimators'-> 150
5. 'my_classifier__max_depth'-> 200

In [ ]:
print('best score {}'.format(pipe_clf_val.best_score_))

best score 0.8520841487993641


# Conclusion:
XGBClassifier gives more and more accuracy 
which is the best one until now
🙉🙉🙉🙉🙉🙉🙉🙉

In [ ]:
import pandas as pand
submission = pand.DataFrame()
submission['id'] = df_test_id

submission['label'] = pipe_clf_val.predict_proba(df_test['text_clean'])[:,1]

submission.to_csv('baysien_search_xg_val.csv', index=False)

# Logistic Regression with character vectorization using BayesSearchCV

In [ ]:
%%time
pipe_lg_ch = Pipeline([
                 ("tfidf", TfidfVectorizer(analyzer="char")),
                 ('my_classifier', LogisticRegression())
                 ])

params_lg_ch = {
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
    'my_classifier__penalty' : ['l1', 'l2'],
    'my_classifier__C' : np.logspace(-4, 4, 20),
    'my_classifier__solver' : ['liblinear']
}


pipe_clf_lg_ch = BayesSearchCV(
    pipe_lg_ch, params_lg_ch, cv=5, verbose=1, n_jobs=2,

    n_iter=50,
    scoring='roc_auc')

pipe_clf_lg_ch.fit(x_train, y_train)


print('best score {}'.format(pipe_clf_lg_ch.best_params_))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

* we make a pipline contains {our vectorizer:tfidf (applid on the character),and the model(classifier):LogisticRegression}
* we played wuth the hyperparameters :
1. max_df->tfidf
2. min_df->tfidf
3. penalty->classifier
4. C ->classifier
5. solver->classifier

* in the BayesSearchCV -> {crossvalidation=5, no. of itreration =50, and our metric ->roc_auc}

best score
1. 'my_classifier__C'-> 0.0001 
2. 'my_classifier__penalty'-> 'l2'
3. 'my_classifier__solver'-> 'liblinear'
4. 'tfidf__max_df'-> 0.3
5. 'tfidf__min_df'-> 42

In [ ]:
print('best score {}'.format(pipe_clf_lg_ch.best_score_))

best score 0.5334006757097621


# Conclusion:
Again, character vectorizer gives very pooooooor accuracy 😞😞😞😞😞😞 

In [ ]:
import pandas as pand
submission = pand.DataFrame()
submission['id'] = df_test_id

submission['label'] = pipe_clf_lg_ch.predict_proba(df_test['text_clean'])[:,1]

submission.to_csv('baysien_search_lg_ch.csv', index=False)

# Logistic Regression with word vectorization using BayesSearchCV

In [ ]:
%%time
pipe_lg_wd = Pipeline([
                 ("tfidf", TfidfVectorizer(analyzer="word")),
                 ('my_classifier', LogisticRegression())
                 ])

params_lg_wd = {
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
    'my_classifier__penalty' : ['l1', 'l2'],
    'my_classifier__C' : np.logspace(-4, 4, 20),
    'my_classifier__solver' : ['liblinear']
}


pipe_clf_lg_wd = BayesSearchCV(
    pipe_lg_wd, params_lg_ch, cv=5, verbose=1, n_jobs=2,
    # number of random trials
    n_iter=50,
    scoring='roc_auc')

# here we still use X_train; but the grid search model
# will use our predefined split internally to determine 
# which sample belongs to the validation set
pipe_clf_lg_wd.fit(x_train, y_train)


print('best score {}'.format(pipe_clf_lg_wd.best_params_))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

* we make a pipline contains {our vectorizer:tfidf (applid on the word),and the model(classifier):LogisticRegression}
* we played wuth the hyperparameters :
1. max_df->tfidf
2. min_df->tfidf
3. penalty->classifier
4. C ->classifier
5. solver->classifier

* in the BayesSearchCV -> {crossvalidation=5, no. of itreration =50, and our metric ->roc_auc}

best score 
1. 'my_classifier__C'-> 1.623776739188721
2. 'my_classifier__penalty'-> 'l2'
3. 'my_classifier__solver'-> 'liblinear'
4. 'tfidf__max_df'-> 0.3
5. 'tfidf__min_df'-> 13

In [ ]:
print('best score {}'.format(pipe_clf_lg_wd.best_score_))

best score 0.8600119182982701


# Conclusion:
Here, again 
the word vecrorization gives the best accuracy (until now)
Logistic regression with word vecrorization is better than with XGBClassifier

In [ ]:
import pandas as pand
submission = pand.DataFrame()
submission['id'] = df_test_id

submission['label'] = pipe_clf_lg_wd.predict_proba(df_test['text_clean'])[:,1]

submission.to_csv('baysien_search_lg_wd.csv', index=False)

# Logistic Regression with validation on the word using BayesSearchCV

we will use our predefined split internally to determine which sample belongs to the validation set

In [ ]:
%%time
pipe_lg_val = Pipeline([
                 ("tfidf", TfidfVectorizer(analyzer="word")),
                 ('my_classifier', LogisticRegression())
                 ])

params_lg_val = {
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
    'my_classifier__penalty' : ['l1', 'l2'],
    'my_classifier__C' : np.logspace(-4, 4, 20),
    'my_classifier__solver' : ['liblinear']
}


pipe_clf_lg_val = BayesSearchCV(
    pipe_lg_val, params_lg_val, cv=pd, verbose=1, n_jobs=2,
    n_iter=10,
    scoring='roc_auc')


pipe_clf_lg_val.fit(x_train, y_train)


print('best score {}'.format(pipe_clf_lg_val.best_params_))

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
best score OrderedDict([('my_classifier__C', 0.615848211066026), ('my_classifier__penalty', 'l2'), ('my_classifier__solver', 'liblinear'), ('tfidf__max_df', 0.3), ('tfidf__min_df', 15)])
CPU times: total: 3.64 s
Wall time: 12.2 s


* we make a pipline contains {our vectorizer:tfidf (applid on the validation set with word),and the model(classifier):LogisticRegression}
* we played wuth the hyperparameters :
1. max_df->tfidf
2. min_df->tfidf
3. penalty->classifier
4. C ->classifier
5. solver->classifier

* in the BayesSearchCV -> {crossvalidation=pd, no. of itreration =10, and our metric ->roc_auc}

best score 
1. 'my_classifier__C -> 0.615848211066026 
2. 'my_classifier__penalty'-> 'l2'
3. 'my_classifier__solver'-> 'liblinear'
4. 'tfidf__max_df'-> 0.3
5. 'tfidf__min_df'-> 15

In [ ]:
print('best score {}'.format(pipe_clf_lg_val.best_score_))

best score 0.8684416427604911


#Conclusion:
Voilaaaaaaaaaaaa 😇😇😇😇😇
Logistic regression on validation set of the word ranked the 1st 
👏👏👏👏👏👏👏👏👏👏 

In [ ]:
import pandas as pand
submission = pand.DataFrame()
submission['id'] = df_test_id

submission['label'] = pipe_clf_lg_val.predict_proba(df_test['text_clean'])[:,1]

submission.to_csv('baysien_search_lg_val.csv', index=False)

# Apply lemmatizer (another type of preprocessing)

There is a rule-based process of word form conversion where word-suffixes are truncated irrespective of whether the root word is an actual word in the language dictionary so we applied lemmatizer
 to convert tokens to their root form

steps:

---




        * remove any html tags (< /br> often found)

        * Keep only ASCII + European Chars and whitespace, no digits

        * remove single letter chars

        * convert all whitespaces (tabs etc.) to single wspace
           if not for embedding (but e.g. tdf-idf):

        * all lowercase

        * remove stopwords, punctuation and stemm

In [ ]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
 
# Create WordNetLemmatizer object
wnl = WordNetLemmatizer()

nltk.download('punkt')
nltk.download('stopwords')

limitization = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))


def clean_text_limit(text, for_embedding=False):
    """ steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [
            limitization.lemmatize(word) for word in words_tokens_lower if word not in stop_words
        ]

    text_clean = " ".join(words_filtered)
    return text_clean

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LAB\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LAB\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LAB\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Clean Comments


In [ ]:
df_train["text_clean_lim"] = df_train.loc[df_train["text"].str.len() > 20, "text"]
df_train["text_clean_lim"] = df_train["text_clean_lim"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x
)

Remove the null or the spaces

In [ ]:
df_train = df_train[(df_train["text_clean_lim"] != "") & (df_train["text_clean_lim"] != "null")]

df_train = df_train.dropna(axis="index", ).reset_index(drop=True)

In [ ]:
# Clean texts for test data
df_test["text_clean_lim"] = df_test.loc[df_test["text"].str.len() > 0, "text"]
df_test["text_clean_lim"] = df_test["text_clean_lim"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x
)

# Split the data to x & Y

In [ ]:
# Sample data - 25% of data to test set
x_train1 = df_train["text_clean_lim"]
y_train1 = df_train["label"]
x_test1=df_test["text_clean_lim"]

print(x_train1.shape)
print(y_train1.shape)
print(x_test1.shape)

(59758,)
(59758,)
(59151,)


# RandomForest to word using BayesSearchCV

In [ ]:
%%time

# feature creation and modelling in a single function
pipe2 = Pipeline([("tfidf", TfidfVectorizer(analyzer="word")), 
                  ("my_classifier", RandomForestClassifier())])

# define parameter space to test # runtime 35min
params2 = {
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
    "my_classifier__n_estimators": [30, 40, 50],  
    "my_classifier__max_depth":[30,50,100]       
}
# it is quite slow so we do 4 for now
pipe_clf2 = BayesSearchCV(
    pipe2, params2, n_jobs=-1, scoring="roc_auc", n_iter=10)
pipe_clf2.fit(x_train1, y_train1)


print('best score {}'.format(pipe_clf2.best_params_))

best score OrderedDict([('my_classifier__max_depth', 100), ('my_classifier__n_estimators', 40), ('tfidf__max_df', 0.3), ('tfidf__min_df', 41)])
CPU times: total: 22 s
Wall time: 2min 2s


* we make a pipline contains {our vectorizer:tfidf (applid on word),and the model(classifier):RandomForest}
* we played wuth the hyperparameters :
1. max_df->tfidf
2. min_df->tfidf
3. mx_depth->classifier

* in the BayesSearchCV -> { no. of itreration =10,
 and our metric ->roc_auc}

best score 
1. 'my_classifier__max_depth'-> 100
2. 'my_classifier__n_estimators'-> 40 
3. 'tfidf__max_df'-> 0.3
4. 'tfidf__min_df'-> 41



In [ ]:
print('best score {}'.format(pipe_clf2.best_score_))

best score 0.8121652483370484


# Conclusion:
RandomForest to the word vectorization gives good accuracy but not the best 😲😲😲😲

# Logistic Regression with validation on word using BayesSearchCV

we will use our predefined split internally to determine which sample belongs to the validation set

In [ ]:
%%time
pipe_lg_val1 = Pipeline([
                 ("tfidf", TfidfVectorizer(analyzer="word")),
                 ('my_classifier', LogisticRegression())
                 ])

params_lg_val1 = {
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
    'my_classifier__penalty' : ['l1', 'l2'],
    'my_classifier__C' : np.logspace(-4, 4, 20),
    'my_classifier__solver' : ['liblinear']
}


pipe_clf_lg_val1 = BayesSearchCV(
    pipe_lg_val1, params_lg_val1, cv=pd, verbose=1, n_jobs=2,
    n_iter=10,
    scoring='roc_auc')



pipe_clf_lg_val1.fit(x_train1, y_train1)


print('best score {}'.format(pipe_clf_lg_val1.best_params_))

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
best score OrderedDict([('my_classifier__C', 29.763514416313132), ('my_classifier__penalty', 'l2'), ('my_classifier__solver', 'liblinear'), ('tfidf__max_df', 0.3), ('tfidf__min_df', 22)])
CPU times: total: 4.09 s
Wall time: 16.5 s


* we make a pipline contains {our vectorizer:tfidf (applid validation set of word),and the model(classifier):Logistic regression}
* we played wuth the hyperparameters :
1. max_df->tfidf
2. min_df->tfidf
3. panlty->classifier
4. C->classifier
5. solver->classifier

* in the BayesSearchCV -> {cv=pd no. of itreration =10,
 and our metric ->roc_auc}

best score 
1. 'my_classifier__C'-> 29.763514416313132
2. 'my_classifier__penalty'-> 'l2'
3. 'my_classifier__solver'-> 'liblinear'
4. 'tfidf__max_df'-> 0.3
5. 'tfidf__min_df'-> 22

In [ ]:
print('best score {}'.format(pipe_clf_lg_val1.best_score_))

best score 0.8596996309711372


# Conclusion:
Still Logistic regression is the best model until now 
and it applied the best on stemmer with validation set on the word
👏👏👏👏👏👏👏👏👏👏👏👏

# XGBClassifier on word on validation using RandomSearchCV

In [ ]:
%%time
pipe1l = Pipeline([
                 ("tfidf", TfidfVectorizer(analyzer="word",ngram_range=(1, 3), norm="l2")),
                 ('my_classifier', XGBClassifier())
                 ])

params1l = {
#     "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
    'my_classifier__n_estimators': [100, 200],  
    'my_classifier__max_depth':[200, 300]
}


pipe_clf1l = RandomizedSearchCV(
    pipe1l, params1l, cv=pd, verbose=1, n_jobs=1,
    # number of random trials
    n_iter=4,
    scoring='roc_auc')

# here we still use X_train; but the grid search model
# will use our predefined split internally to determine 
# which sample belongs to the validation set
pipe_clf1l.fit(x_train1, y_train1)


print('best score {}'.format(pipe_clf1l.best_params_))

Fitting 1 folds for each of 4 candidates, totalling 4 fits
best score {'tfidf__min_df': 32, 'tfidf__max_df': 0.3, 'my_classifier__n_estimators': 200, 'my_classifier__max_depth': 300}
CPU times: total: 1h 32min 24s
Wall time: 17min 46s


* we make a pipline contains {our vectorizer:tfidf (applid validation set of word),and the model(classifier):XGBClassifier}
* we played wuth the hyperparameters :
1. max_df->tfidf
2. min_df->tfidf
3. max_depth->classifier
4. no.estimator->classifier


* in the RandomSearchCV -> {cv=pd no. of itreration =4,
 and our metric ->roc_auc}

best score 
1. 'tfidf__min_df'-> 32
2. 'tfidf__max_df'-> 0.3
3. 'my_classifier__n_estimators'-> 200
5. 'my_classifier__max_depth'-> 300




In [ ]:
print('best score {}'.format(pipe_clf1l.best_score_))

best score 0.8470363970520615


#Conclusion:
XGBClassifier gives good but not best accuracy

In [ ]:
import pandas as pand
submission = pand.DataFrame()
submission['id'] = df_test_id

submission['label'] = pipe_clf1l.predict_proba(df_test['text_clean'])[:,1]

submission.to_csv('baysien_search_lg_val_l.csv', index=False)

#Conclusion:
the overall obesrvation:
* RandomForest gives lower accuracy (it is not good fit for our data set)
* XGBClassifier gives higher accuracy than RandomForest but not the best 
* Logistic regression gives the best accuracy
* lemitizer or stemmer did not make huge differences

# The winner is the Logistic Regression 👏👏👏👏👏👏👏👏👏👏👏👏👏👏👏


#🌈 What is the difference between Character n-gram and Word n-gram? Which one tends to suffer more from the OOV issue?

Word n-gram model : represent unique word sequence of length n as
feature .
Character n-gram model : represent unique character sequence of length n as
feature
(in another way: n-gram means a sequence of n words,a continuous sequence of characters of the specified length while Character n-grams splits each word based on characters but word n-grams splits the text based on words)


Word n-gram tends to suffer more from the OOV issue as:

Out Of Vocabulary (OOV) words refer to the new words(do not exist in the vocabulary) which are faced at testing. Hence, these methods fail in handling OOV words.

[link text](https://reader.elsevier.com/reader/sd/pii/S1877042813041918?token=3E1F8BE1DC047E8C3B89A98E9A583DE265B6477EF7E4DC237DACFA7F133D2A53A3592532CD63666A37288E8E65F64116&originRegion=eu-west-1&originCreation=20220314111053)

#🌈 What is the difference between stop word removal and stemming? Are these techniques language-dependent?

stop words are high frequency words that have small weight and are can not help the retrieval process. 

Stemming remove morphological affixes from words and these techniques are language dependant.

but both are commonly used method in indexing.

#🌈 Is tokenization techniques language dependent? Why?

Tokenization is splitting a phrase, sentence, paragraph, or even a text into smaller units, it should know the meaning of each word to split it ,depending on language (so, it is language dependent).
There are various tokenization techniques available which can be applicable based on the language and purpose of modeling.

Major techniques for tokenizing are:

Split, Spacy, Gensim, Regular Expression, NLTK (Natural Language Toolkit) library, Dictionary Based Tokenization, Rule Based Tokenization, Penn TreeBank Tokenization, and many others...


[link text](https://medium.com/unpackai/natural-language-processing-tokenization-and-numericalization-63c2df20d917)

# 🌈 What is the difference between count vectorizer and tf-idf vectorizer? Would it be feasible to use all possible n-grams? If not, how should you select them?



In CountVectorizer, we only count the number of a word's times that appears in the document . this ends up in ignoring rare words which may help in processing our data more efficiently.

In TfidfVectorizer, we consider all the document (not for each word)weightage of a word , it weights the word counts by a measure of how often they appear in the documents.(it helps us in dealing with most frequent words)


TF-IDF is better than Count Vectorizers as it is not only focuses on the frequency of words present in the corpus but also provides the importance of the words. We can then remove  the less important words for analysis.

No, it Wouldn't be feasible to use all possible n-grams especially if it is of a significant a size

As it depends on the application and the dataset. In some applications, it is important to deal with some combinations of words together as they are together, have a big effect( as sentiment analysis problem)

but, TfidfVectorizer and CountVectorizer both are methods for converting text data into vectors as model can process only numerical data.

[link text](https://www.quora.com/What-is-the-difference-between-TfidfVectorizer-and-CountVectorizer-1)
